In [25]:
from scipy.io import loadmat
import os
import scipy.sparse as sp

In [23]:
## CSTR
file_name = "datasets/cstr.mat"
cstr = loadmat(file_name)

## WebKB4
## WebACE
file_name = "datasets/WebACE.mat"
WebACE = loadmat(file_name)

## NG10
## NG20
file_name = "datasets/NG20.mat"
NG20 = loadmat(file_name)

## RCV1
file_name = "datasets/RCV1_ori.mat"
RCV1 = loadmat(file_name)

## Reviews
file_name = "datasets/reviews.mat"
reviews = loadmat(file_name)

## Sports
file_name = "datasets/sports.mat"
sports = loadmat(file_name)

## Classic3
file_name = "datasets/classic3.mat"
classic3 = loadmat(file_name)

## Classic4
file_path = os.path.join('datasets', "classic4.txt")
nb_row = 7095
nb_col = 5896
classic4 = sp.lil_matrix((nb_row, nb_col))



In [24]:
X

<7095x5896 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [15]:
matlab_dict['A']

<3891x4303 sparse matrix of type '<class 'numpy.float64'>'
	with 176347 stored elements in Compressed Sparse Column format>